In [11]:
import imaplib
import email
from email.header import decode_header

# Login credentials
username = 'xing.di@protagolabs.com'
password = 'xxxxxxxxx!'

# Create an IMAP4 class with SSL 
mail = imaplib.IMAP4_SSL('imap.gmail.com')

# Authenticate
mail.login(username, password)

# Select the mailbox you want to check (INBOX)
mail.select('inbox')

# Search for all unread emails
status, messages = mail.search(None, '(UNSEEN)')

if status == 'OK':
    # Convert the result list to a list of email IDs
    messages = messages[0].split()
    print(f"You have {len(messages)} unread email(s).")
    
    for mail_id in messages:
        # Fetch each email's metadata
        status, data = mail.fetch(mail_id, '(RFC822)')
        
        if status == 'OK':
            # Parse the email's bytes and convert to a message
            message = email.message_from_bytes(data[0][1])
            
            # Decode the email subject
            subject, encoding = decode_header(message['Subject'])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding or 'utf-8')
                
            print("Subject:", subject)

# Don't forget to logout
mail.logout()


error: b'[AUTHENTICATIONFAILED] Invalid credentials (Failure)'

In [12]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import os.path
import base64
import email

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def get_service():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('gmail', 'v1', credentials=creds)
    return service

def list_messages(service, user_id='me', label_ids=None, query=None):
    """List all Messages of the user's mailbox matching the query."""
    try:
        response = service.users().messages().list(userId=user_id, labelIds=label_ids, q=query).execute()
        messages = response.get('messages', [])

        if not messages:
            print("No messages found.")
        else:
            print("Message snippets:")
            for message in messages[:5]:  # Fetch first 5 messages for demo
                msg = service.users().messages().get(userId=user_id, id=message['id'], format='raw').execute()
                
                # Decode the email's body
                msg_str = base64.urlsafe_b64decode(msg['raw'].encode('ASCII'))
                mime_msg = email.message_from_bytes(msg_str)

                print(mime_msg.get_payload())

    except Exception as error:
        print(f'An error occurred: {error}')

def main():
    service = get_service()
    # Call the Gmail API
    list_messages(service, query='is:unread')

if __name__ == '__main__':
    main()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1052714194943-nq2t2qep5lne624k0mj4dhnajjckfmlt.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A35369%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=bgsPrJ6QUmbjlIFFLdCDZPOEdtaS22&access_type=offline


KeyboardInterrupt: 